In [2]:
import requests
from collections import defaultdict
import torch
print('torch version', torch.__version__)
from torch import nn, zeros, cat
from torch.autograd import Variable
import argparse
import string
import numpy as np
import re
import matplotlib.pyplot as plt
import random
from itertools import cycle
import json
import warnings
warnings.filterwarnings("ignore")

torch version 1.3.1


In [3]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


#### Common functions

In [0]:
from_list = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюяАБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ'
to_list = 'a,b,v,g,d,e,jo,zh,z,i,y,k,l,m,n,o,p,r,s,t,u,f,kh,c,ch,sh,shh,jhh,ih,jh,eh,ju,ja,A,B,V,G,D,E,JO,ZH,Z,I,Y,K,L,M,N,O,P,R,S,T,U,F,KH,C,CH,SH,SHH,JHH,IH,JH,EH,JU,JA'.split(',')

In [0]:
def transliterate(string):
    transliterated_string = ''
    for symbol in string:
        if symbol in from_list:
            index = from_list.index(symbol)
            transliterated_string += to_list[index]
        else:
            transliterated_string += symbol
    return transliterated_string

In [0]:
def detransliterate(string):
    detransliterated_string = ''
    index_of_symbol = 0
    while index_of_symbol < len(string):
        was_combination = 0
        for i in range(min(3, len(string)-index_of_symbol), 0, -1):
            if string[index_of_symbol:index_of_symbol+i] in to_list:
                index = to_list.index(string[index_of_symbol:index_of_symbol+i])
                detransliterated_string += from_list[index]
                index_of_symbol += i
                was_combination = 1
                break
        if not was_combination:
            detransliterated_string += string[index_of_symbol]
            index_of_symbol += 1
    return re.findall('[^a-zA-Z]+', detransliterated_string)[0]

In [0]:
# Turning a string into a tensor
def string_to_tensor(string, symbols):
    tensor = torch.zeros(len(string)).to(torch.int64)
    for index in range(len(string)):
        try:
            tensor[index] = symbols.index(string[index])
        except ValueError:
            continue
    return Variable(tensor)

In [0]:
def draw_graph(arguments, params):
    cycol = cycle('bgrcmk')
    fig = plt.figure(figsize=(10, 6))
    ax = fig.add_axes([0.1, 0.1, 1.8, 1.8])
    arguments['model_name'] = 'GRU_' + arguments.model_name
    arguments['type_of_model'] = 0
    arguments['size_of_hidden_layer'] = 550
    arguments['number_of_layers'] = 2
    arguments['rate_of_learning'] = 0.005
    arguments['batch_size'] = 400
    arguments['size_of_learned_sequence'] = 250
    arguments['bias'] = True
    arguments['dropout'] = 0
    arguments['output_bias'] = True
    arguments['bidirectional'] = False
    labels = ['Adam', 'SGD', 'RMSProp']
    for i, size in enumerate([0, 4, 5]):
        arguments['optimizer'] = size
        iterations, losses = train_model(arguments, params)
        ax.plot(iterations, losses, color=next(cycol), label=labels[i], linewidth=3)
    ax.legend(fontsize=30)
    ax.set_title('Cross Entropy Loss Change Over time (LSTM)', fontsize=30)
    ax.set_xlabel('Iteration', fontsize=30)
    ax.set_ylabel('Cross Entropy Loss', fontsize=30)
    plt.tick_params(labelsize=20)
    plt.savefig('loss_function.png', bbox_inches = 'tight', fontsize=30)
    # plt.show()

In [0]:
# with open('/content/drive/My Drive/horoscopes.txt', 'r', encoding='utf-8') as f:
#     texts = f.read().split('\n')

In [0]:
# def parse_all_wordforms(word):
#     url = f'https://ru.wiktionary.org/wiki/{word}'
#     parsed_html = requests.get(url).text
#     st = set()
#     table = re.findall('<table style="float:right; clear:right; margin-left:0.5em; margin-bottom:0.5em; border: 1px solid #6699CC; border-collapse:collapse; max-width:300px;" cellpadding="3" rules="all">(.*?)</table>', parsed_html, re.DOTALL)[0]
#     for elem in re.findall('<td bgcolor="#ffffff">(.*?)</td>', table, re.DOTALL):
#         if '<br />' in elem:
#             for inner_elem in elem.split('<br />'):
#                 st.add(re.sub('́', '', inner_elem.strip()))
#         else:
#             if '<' in elem:
#                 continue
#             st.add(re.sub('́', '', elem.strip()))
#     st -= {'—', '&#160;—&#32;'}
#     return list(st)

In [0]:
# zodiac_sign_to_wordforms = {}
# for znak in ['овен', 'телец', 'близнец', 'рак', 'лев', 'дева', 'весы', 'скорпион', 'стрелец', 'козерог', 'водолей', 'рыба']:
#     zodiac_sign_to_wordforms[znak] = parse_all_wordforms(znak)
# wordforms_to_zodiac_sign = {value_elem: key for key, value in zodiac_sign_to_wordforms.items() for value_elem in value}

In [0]:
# month_to_wordforms = {}
# for month in 'Январь	Февраль	Март	Апрель	Май	Июнь	Июль	Август	Сентябрь	Октябрь	Ноябрь	Декабрь'.lower().split():
#     month_to_wordforms[month] = parse_all_wordforms(month)
# wordforms_to_month = {value_elem: key for key, value in month_to_wordforms.items() for value_elem in value}

In [0]:
# MONTH = 'цясем'
# ZODIAC = 'каидоз'
# YEAR = 'дог'
# DAY = 'ьнед'
# NUMBER = 'ремон'

In [0]:
# converted_month = {key: MONTH for key in month_to_wordforms}
# converted_zodiac_sign = {key: ZODIAC for key in zodiac_sign_to_wordforms}

In [0]:
# def encode_input_text(text):
#     text = re.sub('\d{4,4}', YEAR, text)
#     text = re.sub('\d{1,2}', DAY, text)
#     text = re.sub('\d+', NUMBER, text)
#     text = re.sub('Подробнее\s+$', '', text)
#     converted_text = []
#     for word in text.split():
#         tmp = word.lower().strip(string.punctuation)
#         if tmp in wordforms_to_month:
#             ind = word.lower().index(tmp[0])
#             word = word[:ind] + converted_month[wordforms_to_month[tmp]] + word[ind + len(tmp):]
#         elif tmp in wordforms_to_zodiac_sign:
#             ind = word.lower().index(tmp[0])
#             word = word[:ind] + converted_zodiac_sign[wordforms_to_zodiac_sign[tmp]] + word[ind + len(tmp):]
#         converted_text.append(word)
#     text = ' '.join(converted_text).strip()
#     return text

In [0]:
# with open('/content/drive/My Drive/converted_horoscopes.txt', 'w', encoding='utf-8') as f:
#     for text in texts:
#         tmp = encode_input_text(text)
#         if tmp:
#             f.write(tmp + '\n')

In [0]:
# def decode_output_text(text, params):
#     decoded_words = []
#     for word in text.split():
#         tmp = word.lower().strip(string.punctuation)
#         if not tmp or tmp[0] not in word.lower():
#             continue
#         ind = word.lower().index(tmp[0])
#         if detransliterate(tmp) == ZODIAC:
#             word = word[:ind] + params['cur_zodiac_sign'] + word[ind + len(tmp):]
#         elif detransliterate(tmp) == DAY:
#             word = word[:ind] + params['cur_day'] + word[ind + len(tmp):]
#         elif detransliterate(tmp) == MONTH:
#             word = word[:ind] + params['cur_month'] + word[ind + len(tmp):]
#         elif detransliterate(tmp) == YEAR:
#             word = word[:ind] + params['cur_year'] + word[ind + len(tmp):]
#         elif detransliterate(tmp) == NUMBER:
#             return random.randint(1, 9)
#         decoded_words.append(word)
#     text = ' '.join(decoded_words)
#     return text

#### Training

In [0]:
class CharModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, batch_size=1, type_of_model=0,
                 num_layers=2, bias=True, dropout=0, bidirectional=False, output_bias=True):
        super().__init__()        
        # initializing attributes
        # the number of expected features in the input
        self.input_size = input_size

        # the number of features in the hidden state
        self.hidden_size = hidden_size

        # the number of features in the output
        self.output_size = output_size

        # set batch_size
        self.batch_size = batch_size

        # type of model (LSTM/LSTMCell/GRU)
        self.type_of_model = type_of_model

        # stacked LSTM/GRU or not? if yes, than how many layers?
        self.num_layers = num_layers
        #if self.type_of_model == 1: # LSTMCell
        #    self.num_layers = 1

        self.bias = bias

        # dropout parameter for LSTM and GRU
        self.dropout = dropout

        # bidirectional LSTM or GRU?
        self.bidirectional = bidirectional

        # bias in Linear layer
        self.output_bias = output_bias

        # define metrics
        self.metrics = nn.CrossEntropyLoss()

        # set optimizer to None
        self.optimizer = None

        # define embedding for input
        self.encoder = nn.Embedding(num_embeddings=self.input_size, embedding_dim=self.hidden_size)

        # define model (LSTM/LSTMCell/GRU)
        if self.type_of_model == 0:
            self.model = nn.LSTM(input_size=self.hidden_size, hidden_size=self.hidden_size, \
                                 num_layers=self.num_layers, bias=self.bias, \
                                 dropout=self.dropout, bidirectional=self.bidirectional)
        elif self.type_of_model == 1:
            self.model = nn.LSTMCell(input_size=self.hidden_size, hidden_size=self.hidden_size, \
                                     bias=self.bias)
        elif self.type_of_model == 2:
            self.model = nn.GRU(input_size=self.hidden_size, hidden_size=self.hidden_size, \
                                num_layers=self.num_layers, bias=self.bias, \
                                dropout=self.dropout, bidirectional=self.bidirectional)
        else:
            raise ValueError("Unknown type of model's number")
        if self.bidirectional:
            self.decoder = nn.Linear(2 * self.hidden_size, self.output_size, bias=output_bias)
        else:
            self.decoder = nn.Linear(self.hidden_size, self.output_size, bias=output_bias)

    def forward(self, input_vector, hidden_state):
        batch_size = input_vector.shape[0]
        encoded_vector = self.encoder(input_vector)

        # time dimension X batch dimension X feature dimension
        # timesteps X batch_size X features
        # one batch per epoch => first dimension: 1

        output_vector, hidden_state = self.model(encoded_vector.view(1, batch_size, -1), hidden_state)

        # bidir lstm
        if self.bidirectional:
            forward_output_vector, backward_output_vector = \
                output_vector[:1, :, :self.hidden_size], output_vector[0:, :, self.hidden_size:]
            output_vector = cat((forward_output_vector, backward_output_vector), dim=-1)

        # view is an analogue of numpy reshape
        output_vector = self.decoder(output_vector.view(batch_size, -1))
        return output_vector, hidden_state

    def hidden_state_initialization(self):
        # GPU
        if self.type_of_model == 2:
            # return only hidden state's vector
            if self.bidirectional:
                return cat((Variable(zeros(self.num_layers, self.batch_size, self.hidden_size)),
                            Variable(zeros(self.num_layers, self.batch_size, self.hidden_size))))
            else:
                return Variable(zeros(self.num_layers, self.batch_size, self.hidden_size))
        # LSTM and LSTMCell
        # return hidden state and cell's vectors
        if self.bidirectional:
            return (cat((Variable(zeros(self.num_layers, self.batch_size, self.hidden_size)),
                         Variable(zeros(self.num_layers, self.batch_size, self.hidden_size)))),
                    cat((Variable(zeros(self.num_layers, self.batch_size, self.hidden_size)),
                         Variable(zeros(self.num_layers, self.batch_size, self.hidden_size)))))
        else:
            return (Variable(zeros(self.num_layers, self.batch_size, self.hidden_size)),
                    Variable(zeros(self.num_layers, self.batch_size, self.hidden_size)))

In [0]:
def count_loss(char_model, input_symbols, correct_output_symbols):
    hidden_state = char_model.hidden_state_initialization()

    if arguments.get('cuda'):
        if arguments.get('type_of_model') == 0:
            hidden_state = (hidden_state[0].cuda(), hidden_state[1].cuda())
        else:
            hidden_state = hidden_state.cuda()

    # We need to zero out the gradients because 
    # we don't want to accumulate them on subsequent backward passes
    char_model.zero_grad()

    loss = 0
    for index in range(arguments.get('size_of_learned_sequence')):
        # forward pass
        output_vector, hidden_state = char_model.forward(input_symbols[:, index], hidden_state)
        loss += char_model.metrics(output_vector.view(arguments.get('batch_size'), -1), \
                                   correct_output_symbols[:, index])
    # backward pass
    loss.backward()

    # optimize parameters
    char_model.optimizer.step()

    # return mean loss
    return loss.item() / arguments.get('size_of_learned_sequence')

In [0]:
def data_for_training(text, symbols):
    # batch_size X size_of_learned_sequence
    input_symbols = torch.LongTensor(arguments.get('batch_size'), arguments.get('size_of_learned_sequence'))
    correct_output_symbols = torch.LongTensor(arguments.get('batch_size'), arguments.get('size_of_learned_sequence'))

    for batch_num in range(arguments.get('batch_size')):
        start_position = np.random.randint(0, len(text) - arguments.get('size_of_learned_sequence'))
        input_symbols[batch_num] = string_to_tensor(text[start_position:start_position + \
                                                    arguments.get('size_of_learned_sequence')], symbols)
        correct_output_symbols[batch_num] = string_to_tensor(text[start_position + 1:start_position + \
                                                             arguments.get('size_of_learned_sequence') + 1], symbols)
    if arguments.get('cuda'):
        input_symbols = input_symbols.cuda()
        correct_output_symbols = correct_output_symbols.cuda()
    
    # return tensors wrapped by Variable
    return Variable(input_symbols), Variable(correct_output_symbols)

In [0]:
def train_model(arguments):
    with open(arguments.get('file'), 'r', encoding='utf-8') as f:
        text_without_translit = f.read()
        text = transliterate(text_without_translit) # , 'ru', reversed=True

    symbols = string.printable
    len_symbols = len(string.printable)

    # char_model
    char_model = CharModel(len_symbols, arguments.get('size_of_hidden_layer'), len_symbols, batch_size=arguments.get('batch_size'),
                           type_of_model=arguments.get('type_of_model'), num_layers=arguments.get('number_of_layers'),
                           bias=arguments.get('bias'), dropout=arguments.get('dropout'), bidirectional=arguments.get('bidirectional'),
                           output_bias=arguments.get('output_bias'))

    #char_model = torch.load("/content/GRU_5000_correct_to_Russian.model")

    if arguments.get('cuda'):
        char_model.cuda()

    # define optimizer
    if arguments.get('optimizer') == 0:
        func = torch.optim.Adam
    elif arguments.get('optimizer') == 1:
        func = torch.optim.Adagrad
    elif arguments.get('optimizer') == 2:
        func = torch.optim.Adamax
    elif arguments.get('optimizer') == 3:
        func = torch.optim.Adadelta
    elif arguments.get('optimizer') == 4:
        func =  torch.optim.SGD
    elif arguments.get('optimizer') == 5:
        func = torch.optim.RMSprop
    else:
        raise ValueError("Unknown optimizer number")
    char_model.optimizer = func(char_model.parameters(), lr=arguments.get('rate_of_learning'))

    losses = []
    iterations = []
    try:
        for iteration in range(arguments.get('number_of_iterations')):
            iterations.append(iteration)
            print("Now iteration", iteration)
            input_symbols, correct_output_symbols = data_for_training(text, symbols)
            loss = count_loss(char_model, input_symbols, correct_output_symbols)
            losses.append(loss)
            if iteration % 100 == 0: # here you can change number of iteration when you print loss
                print('loss:', loss)
                generated_example = sampling(char_model, symbols, arguments)
                print(generated_example)
                output_text = detransliterate(generated_example)

                # transliterate names in tags back to English
                tags = re.findall('<.*?>', output_text)
                for tag in tags:
                    try:
                        output_text = re.sub(re.escape(tag), transliterate(tag), output_text)
                    except:
                        print(tag, transliterate(tag))
                print('output:\n', output_text, '\n')

        # NB: конкретно для этого случая
    except Exception as e:
        print(f"ERROR HAPPENED: {e}")
    print("!!!")
    torch.save(char_model, "LSTM_6500.model")

    # save only the model parameters
    # torch.save(char_model.state_dict(), arguments.get('file')[:-4] + '.model')
    return iterations, losses

#### Generating

In [0]:
def sampling(char_model, symbols, arguments):
    # save previous value
    tmp = char_model.batch_size

    char_model.batch_size = 1
    hidden_state = char_model.hidden_state_initialization()
    char_model.zero_grad()

    # set back previous value
    char_model.batch_size = tmp

    # insert a singleton dimension at the first place and wrap by Variable
    input_symbols = Variable(string_to_tensor(arguments.get('start_of_sequence'), symbols).unsqueeze(0))

    if arguments.get('cuda'):
        if char_model.type_of_model == 0:
            hidden_state = (hidden_state[0].cuda(), hidden_state[1].cuda())
        else:
            hidden_state = hidden_state.cuda()
        input_symbols = input_symbols.cuda()

    for index in range(len(arguments.get('start_of_sequence')) - 1):
        # forward pass
        output_vector, hidden_state = char_model.forward(input_symbols[:, index], hidden_state)

    input_symbols = input_symbols[:, -1]
    generated_text = arguments.get('start_of_sequence')

    for _ in range(arguments.get('size_of_generated_sequence')):
        # forward pass
        output_vector, hidden_state = char_model.forward(input_symbols, hidden_state)

        # divide by temperature so that to get more conservative or more arbitrary samples
        # the lower the temperature, the more conservative output
        # the higher is temperature, the more random output
        # scaling logits before applying softmax

        # we add 0.1 in order to avoid INF
        bias = 0.1

        try:
            # output_vector = torch.softmax(output_vector.div(arguments.get('temperature')), 0)
            # output_vector = torch.exp(output_vector / arguments.get('temperature')).view(-1)
            output_vector = output_vector.data.view(-1).div(arguments.get('temperature')).exp()
        except RuntimeError:
            output_vector = torch.exp(output_vector / (arguments.get('temperature') + bias)).view(-1)

        # The rows of input do not need to sum to one (in which case we use the values as weights)
        # and it doesn't in this case
        index_of_symbol = torch.multinomial(input=output_vector, num_samples=1)[0]

        # print(index_of_symbol)
        # also possible via softmax and argmax prediction but terrible quality
        # probs = torch.softmax(output_vector, 0)
        # index_of_symbol = torch.max(probs, 0)[1].item()

        index_of_symbol = index_of_symbol.data
        if index_of_symbol >= 0 and index_of_symbol < len(symbols):
            generated_text += symbols[index_of_symbol]
        else:
            # in case of unprintable symbol
            generated_text += symbols[np.random.randint(0, len(symbols))] # right border not included
        # generated_symbol
        input_symbols = Variable(string_to_tensor(generated_text[-1], symbols).unsqueeze(0))

        if arguments.get('cuda'):
            input_symbols = input_symbols.cuda()
        # stop when \n is encountered
        # if generated_text[-2:] == '\n\n':
        #     break

    return generated_text

#### Let's do it!

In [0]:
# Algorithms:
# 0 = LSTM
# 1 = LSTMCell
# 2 = GRU

# Gradient optimizatization techniques
# 0 = Adam
# 1 = Adagrad
# 2 = Adamax
# 3 = Adadelta
# 4 = SGD
# 5 = RMSProp

arguments = {
    'file': "/content/drive/My Drive/models/all_pairs.txt",
    'type_of_model': 0,
    'size_of_hidden_layer': 500,
    'number_of_layers': 2,
    'number_of_iterations': 6500,
    'rate_of_learning': 0.005,
    'batch_size': 500,
    'bias': True,
    'dropout': 0,
    'bidirectional': True,
    'output_bias': True,
    'size_of_learned_sequence': 300,
    'optimizer': 0,
    'size_of_generated_sequence': 250,
    'start_of_sequence': 'Вы',
    'temperature': 0.5,
    'draw_graph': 0,
    'model_name': "LSTM.model",
    'cuda': 1
}

if arguments.get('draw_graph') == 0:
    iterations, losses = train_model(arguments)
    with open("LSTM_iters_and_losses.json", 'w', encoding='utf-8') as f:
        json.dump((iterations, losses), f)
else:
    draw_graph(arguments, params)

Now iteration 0
loss: 4.6089180501302085
Выt{T naO a      o o       o b        n    o     ao  o    o or  o o      t o     o       d  eo  o o o      o r  e o o         r zn o      o   o        n     o  noo  odh   oaa no      ou oo    o on   pono       oo  o     m s  lot    na     ch  u l     k
output:
 Выт{Т наО а      о о       о б        н    о     ао  о    о ор  о о      т о     о       д  ео  о о о      о р  е о о         р зн о      о   о        н     о  ноо  од 

Now iteration 1
Now iteration 2
Now iteration 3
Now iteration 4
Now iteration 5
Now iteration 6
Now iteration 7
Now iteration 8
Now iteration 9
Now iteration 10
Now iteration 11
Now iteration 12
Now iteration 13
Now iteration 14
Now iteration 15
Now iteration 16
Now iteration 17
Now iteration 18
Now iteration 19
Now iteration 20
Now iteration 21
Now iteration 22
Now iteration 23
Now iteration 24
Now iteration 25
Now iteration 26
Now iteration 27
Now iteration 28
Now iteration 29
Now iteration 30
Now iteration 31
Now itera

In [95]:
arguments = {
    # /content/drive/My Drive/
    'file': "LSTM_1000.model",
    'start_of_sequence': 'Как жить-то нам дальше?\n',
    'size_of_generated_sequence': 500,
    'temperature': 0.5,
    'cuda': 1
}

char_model = torch.load(arguments.get('file'))
# load only the model parameters
# char_model = CharModel(*args, **kwargs) # but you should know the args parameters of training
# char_model.load_state_dict(torch.load(arguments.get('file')))
# char_model.eval()

detransliterated_text = detransliterate(sampling(char_model, string.printable, arguments))
detransliterated_text = re.sub('\d+', ' ', detransliterated_text)
detransliterated_text = re.sub(' +', ' ', detransliterated_text)
detransliterated_text = '\n'.join([sent.strip() for sent in detransliterated_text.split('\n')])
print(detransliterated_text)

Как жить-то нам дальше?
В предстоявшем законе и воспитании при приезде полковника на Веры по его двери и поместителей от Москвы и после этого пространства с детства после получения воспитания в книге.

Вот этого не слышал, представила его и стал смотреть его, но в этом же сыном в черновиком из картинах.
По поводу издания в г. начальника пристально служил в сторону его приезда Азбуки о написаниях и в Б стр.

Приехал в свое лицо и стал сказать, о котором она видела на него.
Когда он пол


In [0]:
detransliterate('Вы каидоз в субботу с которых можно полностью заблуждаться с ними своими родными близким. Если стремления в личной жизни особенно заметно, по достойной забот должного внимания. Будьте осторожны в полной мере проявлять исполнения своих детей. ьнед числа Вам придется проявить ошибки, раз поставленностью в собственном положении. Однако в отношениях с родными придет не было смысла, занятную прибкой новые перспективы. Сбегите это не слишком домашним построениj')

'В'